In [1]:
import os
import json
import pandas as pd

from tqdm import tqdm
from gensim.models import Word2Vec
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors


import json


with open('/Users/joons/work_directory/python_notebook/melon/KakaoArena/data/train.json' , encoding = 'utf-8' ) as f:
    train = json.load(f)
song_meta = pd.read_json('/Users/joons/work_directory/python_notebook/melon/KakaoArena/data/song_meta.json')
song_meta = song_meta[['id','song_name','artist_name_basket']]

In [2]:
print(len(train))

115071


In [3]:
data = pd.DataFrame(train[:100])
data.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000


### 전처리

In [5]:
from tqdm import tqdm

song_dic = {}
tag_dic = {}

data = train 
for q in tqdm(data):
    song_dic[ str( q['id'] ) ] = q['songs']
    tag_dic[ str( q['id'] ) ] = q['tags']
    
total = list( map( lambda x : list( map(str , x['songs'])) + list( x['tags']) , data))
total = [ x for x in total if len(x) > 1]

100%|██████████| 115071/115071 [00:00<00:00, 648905.26it/s]


<br></br><br></br>

Embedding

In [6]:
w2v_model = Word2Vec( total[: 10000 ] , min_count = 1 , size = 32 , window = 5 ,sg = 1)

In [7]:
w2v_model.wv.vectors[:2]

array([[ 1.7072518 ,  0.30986273, -0.84273595, -1.57937   ,  1.2243936 ,
        -1.2356309 , -0.3445803 , -1.039198  , -0.4941444 ,  1.8235054 ,
         0.6134956 , -2.8927789 , -0.23839399, -0.17302892,  2.226164  ,
        -1.1752524 ,  0.8505482 ,  0.8412116 ,  0.864127  , -1.663487  ,
        -0.07474238, -1.3666345 ,  1.0315909 , -1.2872353 , -0.6383945 ,
        -0.20910506, -0.8298315 ,  1.4533874 ,  0.30448964, -3.2909472 ,
        -0.853124  ,  0.713152  ],
       [ 1.2686348 ,  0.45827404, -0.7100548 , -1.7408212 ,  1.4168236 ,
        -1.4690228 ,  0.5390897 , -0.65708596, -0.2513751 ,  1.2875065 ,
         0.9295601 , -2.6779683 ,  0.45979244, -0.65621537,  1.0561393 ,
        -0.77993125,  0.7051674 ,  0.99514955,  0.43472368, -0.78745717,
        -0.77018297, -1.5746156 ,  0.35138062, -0.76858217,  0.61311364,
        -0.19802369, -1.0858772 ,  1.198889  ,  0.23989329, -2.9667974 ,
        -0.6876013 ,  0.7794777 ]], dtype=float32)

In [8]:
print(len(list(w2v_model.wv.vocab.keys())))


def get_song_info(content):
    return song_meta.loc[song_meta.id == int(content)]

def compare_w2v( content ):
    print(content)
    print('similar_contents')
    for c,sim in w2v_model.wv.similar_by_word(content):
        try :
            c = int(c)
            tmp = get_song_info(c).values[0]
            print(f"song : {tmp[1]} artist : {tmp[2]} song_id :{tmp[0]}")
        except ValueError:
            print(f"tag : {c}")
            pass
        
        

163670


In [9]:
contents = list(w2v_model.wv.vocab.keys())
get_song_info(contents[45])

,id,song_name,artist_name_basket
152422,152422,너무 보고싶어,[어쿠스틱 콜라보]


In [10]:
compare_w2v('152422')

152422
similar_contents
song : 고백 artist : ['정준일'] song_id :59003
song : 그냥 (Just) artist : ['Zion.T', 'Crush'] song_id :339124
song : 언제쯤이면 (Duet.아이유) artist : ['윤현상'] song_id :298103
song : 진심 (Feat. 임세준) artist : ['정키'] song_id :151183
song : 너의 모든 순간 artist : ['성시경'] song_id :295250
song : 스토커 artist : ['10CM'] song_id :449244
song : 이 소설의 끝을 다시 써보려 해 artist : ['한동근'] song_id :543820
song : 시간과 낙엽 artist : ['AKMU (악동뮤지션)'] song_id :609835
song : 거짓말 거짓말 거짓말 artist : ['이적'] song_id :13142
song : 바람이 불었으면 좋겠어 artist : ['길구봉구'] song_id :357489


In [11]:
get_song_info(contents[456])

,id,song_name,artist_name_basket
275121,275121,뿜뿜,[모모랜드 (MOMOLAND)]


In [12]:
compare_w2v('275121')

275121
similar_contents
song : Roller Coaster artist : ['청하'] song_id :138486
song : 주인공 artist : ['선미'] song_id :465654
song : 별이 빛나는 밤 artist : ['마마무 (Mamamoo)'] song_id :678035
song : Heart Shaker artist : ['TWICE (트와이스)'] song_id :421476
song : 사랑을 했다 (LOVE SCENARIO) artist : ['iKON'] song_id :372088
song : 여행 artist : ['볼빨간사춘기'] song_id :346619
song : 비밀정원 artist : ['오마이걸'] song_id :537208
song : LIKEY artist : ['TWICE (트와이스)'] song_id :642
song : Bad Boy artist : ['Red Velvet (레드벨벳)'] song_id :588696
song : 에너제틱 (Energetic) artist : ['Wanna One (워너원)'] song_id :108003


In [ ]:
get_song_info(contents[456])